# Lesson 1 - Simple LLM call using langchain
This is the basic lesson to show you how to invoke an LLM using langchain

In [1]:
# We first install the packages
%pip install langchain langchain-openai langchain-community --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
# We install our envirnment variables helper library
%pip install python-dotenv --quiet

# We load the library and the environment variables
import dotenv
dotenv.load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

In [3]:
# We load the settings
import _settings

# The challenge prompt is the challenge we want our agents to solve
# You can change it in the _settings.py file
# Be sure to restart ^^ your kernel after changing the settings
challenge_prompt = _settings.CHALLENGE_PROMPT
print(challenge_prompt)

Write me a shell script to list the files in the current directory. 
Output the script as markdown code block


In [4]:
# We load the llm language model from _models
from langchain_openai import OpenAI
llm = OpenAI()

# We invoke the language model with the challenge prompt
result = llm.invoke(_settings.CHALLENGE_PROMPT)

# We print the result
print(result)




```
#!/bin/bash

# This script lists all the files in the current directory

echo "Files in current directory:"

ls # lists all the files in the current directory

# To make the output more readable, we can use the -l flag to display the files in a list format

echo "Files in current directory (in list format):"

ls -l # lists all the files in the current directory in a list format

# To make the output more detailed, we can use the -a flag to display all files, including hidden ones

echo "All files in current directory (including hidden files):"

ls -a # lists all files in the current directory, including hidden files

# To display only the files with a specific extension, we can use the * wildcard character

echo "Files with .txt extension:"

ls *.txt # lists all files in the current directory with a .txt extension

# To save the output to a file, we can use the > symbol and specify the file name

ls > file_list.txt # saves the list of files in the current directory to a file name

To see what is going of the wire to the LLM and back, we add a callback handler that prints this out.

In [5]:
# We import the callback handler from _callbacks
from _callbacks import PrettyPrintCallbackHandler
callback = PrettyPrintCallbackHandler()

# Now we can do the same call
# but this time we will use the callback handler
# That will print out the intermediate results
# We set the streaming parameter to True to see how the callback works

# To make sure we always the same result we set the temperature to 0
# Think of reducing the creative randomness of the llm
llm = OpenAI(streaming=True, temperature=0 , callbacks=[callback])
result=llm.invoke(challenge_prompt)



[llm][start] - prompts: Write me a shell script to list the files in the current directory. 
Output the script as markdown code block
[llm][new_token] 


[llm][new_token] ``
[llm][new_token] `

[llm][new_token] #!/
[llm][new_token] bin
[llm][new_token] /bash
[llm][new_token] 


[llm][new_token] #
[llm][new_token]  This
[llm][new_token]  script
[llm][new_token]  lists
[llm][new_token]  all
[llm][new_token]  the
[llm][new_token]  files
[llm][new_token]  in
[llm][new_token]  the
[llm][new_token]  current
[llm][new_token]  directory
[llm][new_token] 


[llm][new_token] #
[llm][new_token]  Change
[llm][new_token]  directory
[llm][new_token]  to
[llm][new_token]  current
[llm][new_token]  directory
[llm][new_token] 

[llm][new_token] cd
[llm][new_token]  .


[llm][new_token] #
[llm][new_token]  Loop
[llm][new_token]  through
[llm][new_token]  all
[llm][new_token]  files
[llm][new_token]  in
[llm][new_token]  current
[llm][new_token]  directory
[llm][new_token] 

[llm][new_token] for
[llm][n

In [6]:
# We print the final result
# And get the same result as before
print(result)



```
#!/bin/bash

# This script lists all the files in the current directory

# Change directory to current directory
cd .

# Loop through all files in current directory
for file in *
do
    # Check if file is a regular file
    if [ -f "$file" ]
    then
        # Print file name
        echo "$file"
    fi
done
```

To run this script, save it as a .sh file and run it in the terminal using the command `./script.sh`.


In [7]:
# Now to avoid we need to this every time in a notebook
# We can put this in a function in _models

# The default model is set in _settings.py
import _models, _settings
# We print the model name from settings
print(_settings.MODEL_NAME)

gpt-4
[chat_model][start] - prompts : Write me a shell script to list the files in the current directory. 
Output the script as markdown code block

[llm][end] - generation Here is the shell script you requested:

```bash
#!/bin/bash
# This script will list all files in the current directory

for file in ./*
do
  if [ -f "$file" ]; then
    echo "$file"
  fi
done
```

This script will loop through all items in the current directory and print out the name of each item that is a file.
content='Here is the shell script you requested:\n\n```bash\n#!/bin/bash\n# This script will list all files in the current directory\n\nfor file in ./*\ndo\n  if [ -f "$file" ]; then\n    echo "$file"\n  fi\ndone\n```\n\nThis script will loop through all items in the current directory and print out the name of each item that is a file.' response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 29, 'total_tokens': 110}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 's

In [ ]:

# We pass this model name to the get_llm helper function
llm = _models.get_llm(_settings.MODEL_NAME,callbacks=[callback])
result=llm.invoke(challenge_prompt)

# We print the final result
print(result)

In [8]:
# For reference you can see the whole helper _models.py file
%pycat _models.py 

from langchain_openai import OpenAI, ChatOpenAI
from langchain_community.chat_models.ollama import ChatOllama

def get_llm(name, callbacks=[]):
    llm = OpenAI(streaming=True, temperature=0 , callbacks=callbacks)

    openai_llm = ChatOpenAI(
        model="gpt-4",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
        callbacks=callbacks
        # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
        # base_url="...",
        # organization="...",
        # other params...
    )

    #if name == "ollama":
    ollama_llm = ChatOllama(
        model="llama3",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
        callbacks=callbacks

        # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
        # base_url="...",
        # organization="...",
        # other params...
    )

    llm = openai_llm
    return llm
